In [411]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML # used to print out pretty pandas dataframes
display(HTML("<style>.container { width:100% !important; }</style>")) # widest display
import matplotlib.dates as dates
import matplotlib.lines as mlines

pd.options.display.max_colwidth = 1000
pd.options.display.width = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 200


%matplotlib inline
plt.style.use('ggplot') 

# specify user/password/where the database is
sqluser = 'eightiesfanjan'
sqlpass = 'squiggle'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

## validate prescriptions

In [402]:
query = query_schema + """
SELECT *
    , (enddate-startdate) AS time
FROM prescriptions
WHERE subject_id = 64 AND hadm_id = 172056 AND
lower(drug) ~ '(hydromorphone|vicodin|oxycodone|oxycontin|percocet|morphine|fentanyl|tramadol|buprenorphine|suboxone|methadone|oxymorphone|opana|meperidine|demerol)' 
ORDER BY startdate
"""
df_clean= pd.read_sql_query(query,con)
df_clean

,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route,time
0,905911,64,172056,232593.0,2143-03-03,2143-03-18,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate,MORP2I,004070,00074176201,2mg Syringe,2-4,mg,1-2,SYR,IV,15 days
1,894908,64,172056,232593.0,2143-03-04,2143-03-04,MAIN,Morphine Sulfate,Morphine Sulfate,Morphine Sulfate,MORP4I,004072,00074125801,4mg Syringe,1-2,mg,0.25-0.5,SYR,IV,0 days
2,894914,64,172056,232593.0,2143-03-04,2143-03-06,MAIN,Hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,00008029501,2MG/ML SYR,1-2,mg,0.5-1,ml,IV,2 days
3,894917,64,172056,232593.0,2143-03-05,2143-03-06,MAIN,Fentanyl Citrate,Fentanyl Citrate,Fentanyl Citrate,FENT2I,041384,11098003002,100MCG/2ML AM,100-200,mcg,2-4,ml,IV,1 days
4,894921,64,172056,232593.0,2143-03-05,2143-03-18,MAIN,Hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,00008029501,2MG/ML SYR,1-2,mg,0.5-1,ml,IV,13 days
5,894928,64,172056,232593.0,2143-03-06,2143-03-18,MAIN,Hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,00008029501,2MG/ML SYR,1,mg,0.5,ml,IV,12 days
6,894927,64,172056,232593.0,2143-03-06,2143-03-18,MAIN,Methadone HCl,Methadone HCl,Methadone HCl,METD10,004240,00054855424,10MG TAB,10,mg,1,TAB,PO,12 days
7,894935,64,172056,NaN,2143-03-09,2143-03-18,MAIN,Hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,00008029501,2MG/ML SYR,1,mg,0.5,ml,IV,9 days
8,894938,64,172056,NaN,2143-03-10,2143-03-18,MAIN,Hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,00008029501,2MG/ML SYR,1,mg,0.5,ml,SC,8 days
9,894945,64,172056,NaN,2143-03-13,2143-03-18,MAIN,Hydromorphone,Hydromorphone,Hydromorphone,HYDR2I,004103,00008029501,2MG/ML SYR,1,mg,0.5,ml,IV,5 days


### Get comorbidities flags

# Playground

### Now filter out for cancer terminal or anoxic brain injury

In [ ]:
"""
    Comorbidities using ICD9 Codes:
	COPD
	Diabetes
	Coronary artery disease
	Congestive heart failure: Systolic
Congestive heart failure:  Diastolic
	End stage renal disease
	End stage liver disease
	Stroke
	Obesity
"""


In [405]:
query = query_schema + """

SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
WHERE (lower(short_title) like '%alpha-1-antitrypsin deficiency%') OR (lower(long_title) LIKE '%antitrypsin%')

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,2734,Alpha-1-antitrypsin def,Alpha-1-antitrypsin deficiency


In [406]:
query = query_schema + """

SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
WHERE (lower(short_title) like '%emphysema%') OR (lower(long_title) LIKE '%emphysema%')

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,4920,Emphysematous bleb,Emphysematous bleb
1,4928,Emphysema NEC,Other emphysema
2,5181,Interstitial emphysema,Interstitial emphysema
3,5182,Compensatory emphysema,Compensatory emphysema
4,7702,NB interstit emphysema,Interstitial emphysema and related conditions
5,99881,Emphysema rsult frm proc,Emphysema (subcutaneous) (surgical) resulting from procedure
6,V813,Screen-bronch/emphysema,Screening for chronic bronchitis and emphysema
7,9587,Traum subcutan emphysema,Traumatic subcutaneous emphysema


In [404]:
query = query_schema + """

SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
WHERE icd9_code = '49122'

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,49122,Obs chr bronc w ac bronc,Obstructive chronic bronchitis with acute bronchitis


In [ ]:
query = query_schema + """
WITH diabetes AS (
    SELECT icd9_code
        , short_title
        , long_title
    FROM d_icd_diagnoses
    WHERE (lower(short_title) like '%diabetes%') OR (lower(long_title) LIKE '%diabetes%')
), COPD AS(
    SELECT ic9_code
        , short_title
        , long_title
    WHERE icd9_code ~ '(49122|2734)' OR
    (lower(short_title) like '%emphysema%') OR (lower(long_title) LIKE '%emphysema%')
)

"""
df_clean= pd.read_sql_query(query,con)
df_clean

#### Anoxic brain

In [350]:
query = query_schema + """
SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
WHERE (lower(short_title) like '%anoxic%') OR (lower(long_title) LIKE '%anoxic%')

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,3481,Anoxic brain damage,Anoxic brain damage


#### opioid icd

In [240]:
query = query_schema + """
SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
WHERE (lower(short_title) like '%opioid%') OR (lower(long_title) LIKE '%opioid%')

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,30400,Opioid dependence-unspec,"Opioid type dependence, unspecified"
1,30401,Opioid dependence-contin,"Opioid type dependence, continuous"
2,30402,Opioid dependence-episod,"Opioid type dependence, episodic"
3,30403,Opioid dependence-remiss,"Opioid type dependence, in remission"
4,30470,Opioid/other dep-unspec,"Combinations of opioid type drug with any other drug dependence, unspecified"
5,30471,Opioid/other dep-contin,"Combinations of opioid type drug with any other drug dependence, continuous"
6,30472,Opioid/other dep-episod,"Combinations of opioid type drug with any other drug dependence, episodic"
7,30473,Opioid/other dep-remiss,"Combinations of opioid type drug with any other drug dependence, in remission"
8,30480,Comb drug dep NEC-unspec,"Combinations of drug dependence excluding opioid type drug, unspecified"
9,30481,Comb drug dep NEC-contin,"Combinations of drug dependence excluding opioid type drug, continuous"


In [241]:
query = query_schema + """
SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
WHERE (lower(short_title) like '%opiate%') OR (lower(long_title) LIKE '%opiate%')

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,E9352,Adv eff opiates,Other opiates and related narcotics causing adverse effects in therapeutic use
1,E9401,Adv eff opiat antagonist,Opiate antagonists causing adverse effects in therapeutic use
2,96509,Poisoning-opiates NEC,Poisoning by other opiates and related narcotics
3,9701,Poison-opiate antagonist,Poisoning by opiate antagonists
4,E8502,Acc poison-opiates NEC,Accidental poisoning by other opiates and related narcotics


In [259]:
query = query_schema + """
SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
-- build icd9 opiate dependence codes (with or without other drugs) from: https://www.ncbi.nlm.nih.gov/books/NBK367628/table/sb202.t4/?report=objectonly
-- but also exclude those that have had poisoning or opidate abuse
WHERE icd9_code  ~ '(30400|30401|30402|30403|30470|30471|30472|30473)' AND 
    icd9_code !~ '(E8502|96509|30550|30551|30552|30553)'

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,30400,Opioid dependence-unspec,"Opioid type dependence, unspecified"
1,30401,Opioid dependence-contin,"Opioid type dependence, continuous"
2,30402,Opioid dependence-episod,"Opioid type dependence, episodic"
3,30403,Opioid dependence-remiss,"Opioid type dependence, in remission"
4,30470,Opioid/other dep-unspec,"Combinations of opioid type drug with any other drug dependence, unspecified"
5,30471,Opioid/other dep-contin,"Combinations of opioid type drug with any other drug dependence, continuous"
6,30472,Opioid/other dep-episod,"Combinations of opioid type drug with any other drug dependence, episodic"
7,30473,Opioid/other dep-remiss,"Combinations of opioid type drug with any other drug dependence, in remission"


### cancer

In [239]:
query = query_schema + """
SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
-- build icd9 cancer codes from: https://www.ncbi.nlm.nih.gov/books/NBK230788/
WHERE lower(short_title) LIKE '%cancer%' OR lower(long_title) LIKE '%cancer%'

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,79582,Elev ca antigen 125,Elevated cancer antigen 125 [CA 125]


In [248]:
query = query_schema + """
SELECT icd9_code
    , short_title
    , long_title
FROM d_icd_diagnoses
-- build icd9 cancer codes from: https://www.ncbi.nlm.nih.gov/books/NBK230788/
WHERE lower(short_title) LIKE '%metastatic%' OR lower(long_title) LIKE '%metastatic%'
e
"""
df_clean= pd.read_sql_query(query,con)
df_clean

,icd9_code,short_title,long_title
0,36314,Metastat dissem retinit,"Disseminated retinitis and retinochoroiditis, metastatic"


### discharge events

In [413]:
query = query_schema + """
SELECT subject_id, hadm_id, text
FROM noteevents
WHERE lower(category) LIKE '%discharge%'

"""
df_clean= pd.read_sql_query(query,con)
df_clean

,subject_id,hadm_id,text
0,22532,167853,"Admission Date: [**2151-7-16**] Discharge Date: [**2151-8-4**]\n\n\nService:\nADDENDUM:\n\nRADIOLOGIC STUDIES: Radiologic studies also included a chest\nCT, which confirmed cavitary lesions in the left lung apex\nconsistent with infectious process/tuberculosis. This also\nmoderate-sized left pleural effusion.\n\nHEAD CT: Head CT showed no intracranial hemorrhage or mass\neffect, but old infarction consistent with past medical\nhistory.\n\nABDOMINAL CT: Abdominal CT showed lesions of\nT10 and sacrum most likely secondary to osteoporosis. These can\nbe followed by repeat imaging as an outpatient.\n\n\n\n [**First Name8 (NamePattern2) **] [**First Name4 (NamePattern1) 1775**] [**Last Name (NamePattern1) **], M.D. [**MD Number(1) 1776**]\n\nDictated By:[**Hospital 1807**]\nMEDQUIST36\n\nD: [**2151-8-5**] 12:11\nT: [**2151-8-5**] 12:21\nJOB#: [**Job Number 1808**]\n"
1,13702,107527,"Admission Date: [**2118-6-2**] Discharge Date: [**2118-6-14**]\n\nDate of Birth: Sex: F\n\nService: MICU and then to [**Doctor Last Name **] Medicine\n\nHISTORY OF PRESENT ILLNESS: This is an 81-year-old female\nwith a history of emphysema (not on home O2), who presents\nwith three days of shortness of breath thought by her primary\ncare doctor to be a COPD flare. Two days prior to admission,\nshe was started on a prednisone taper and one day prior to\nadmission she required oxygen at home in order to maintain\noxygen saturation greater than 90%. She has also been on\nlevofloxacin and nebulizers, and was not getting better, and\npresented to the [**Hospital1 18**] Emergency Room.\n\nIn the [**Hospital3 **] Emergency Room, her oxygen saturation was\n100% on CPAP. She was not able to be weaned off of this\ndespite nebulizer treatment and Solu-Medrol 125 mg IV x2.\n\nReview of systems is negative for the following: Fevers,\nchills, nausea, vomiting, n..."
2,13702,167118,"Admission Date: [**2119-5-4**] Discharge Date: [**2119-5-25**]\n\n\nService: CARDIOTHORACIC\n\nAllergies:\nAmlodipine\n\nAttending:[**Last Name (NamePattern1) 1561**]\nChief Complaint:\n81 yo F smoker w/ COPD, severe TBM, s/p tracheobronchoplasty [**5-5**]\ns/p perc trach [**5-13**]\n\nMajor Surgical or Invasive Procedure:\nbronchoscopy 3/31,4/2,3,[**6-12**], [**5-17**], [**5-19**]\ns/p trachealplasty [**5-5**]\npercutaneous tracheostomy [**5-13**] after failed extubation\ndown size trach on [**5-25**] to size 6 cuffless\n\n\nHistory of Present Illness:\nThis 81 year old woman has a history of COPD. Over the past five\n\nyears she has had progressive difficulties with her breathing.\nIn\n[**2118-6-4**] she was admitted to [**Hospital1 18**] for respiratory failure\ndue\nto a COPD exacerbation. Due to persistent hypoxemia, she\nrequired\nintubation and a eventual bronchoscopy on [**2118-6-9**] revealed marked\n\nnarrowing of the airways on expiration consistent with\..."
3,13702,196489,"Admission Date: [**2124-7-21**] Discharge Date: [**2124-8-18**]\n\n\nService: MEDICINE\n\nAllergies:\nAmlodipine\n\nAttending:[**First Name3 (LF) 898**]\nChief Complaint:\nCOPD exacerbation/Shortness of Breath\n\nMajor Surgical or Invasive Procedure:\nIntubation\narterial line placement\nPICC line placement\nEsophagogastroduodenoscopy\n\n\nHistory of Present Illness:\n87 yo F with h/o CHF, COPD on 5 L oxygen at baseline,\ntracheobronchomalacia s/p stent, presents with acute dyspnea\nover several days, and lethargy. This morning patient developed\nan acute worsening in dyspnea, and called EMS. EMS found patient\ntachypnic at saturating 90% on 5L. Patient was noted to be\ntripoding. She was given a nebulizer and brought to the ER.\n.\nAccording the patient's husband, she was experiencing symptoms\nconsistent with prior COPD flares. Apparently patient was\nwithout cough, chest pain, fevers, chills, orthopnea, PND,\ndysuria, diarrhea, confusion and neck pain. Her husban..."
4,26880,135453,"Admission Date: [**2162-3-3**] Discharge Date: [**2162-3-25**]\n\nDate of Birth: [*